In [1]:
import re
import codecs
import random
from PIL import Image
import numpy as np
import pandas as pd
from keras.preprocessing import image
from keras.models import load_model
from keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)
import os

Using TensorFlow backend.


In [2]:
def calculateDistance(i1, i2):
    return np.sqrt(np.sum((i1-i2)**2))

In [3]:
def find_images_to_find_in(class_):
    match_with_images = []
    city_state_location = []
    class_file = r"iceland_"+class_.replace(" ","_")+"_lat_long_city_state.txt"
    with codecs.open(class_file, 'r', encoding="utf-8-sig") as f:
        for line in f:
            elems = line.strip("\r|\n").split("|")
            storage_location, latitude, longitude, city, state = elems
            image_location = storage_location
            try:
                im1 = Image.open(image_location)
                im2 = im1.resize((299, 299), Image.NEAREST)
                test_image = np.array(im2)
                match_with_images.append(test_image)
                city_state_location.append(storage_location.replace(r"C:\Users\bg532fh\Documents\Personal\Kaggle\Vacation Destination\intel-image-classification\Dataset\\","")+","+city+","+state)
            except Exception as e:
                continue
    return match_with_images,city_state_location

In [5]:
geyser_match_with_images,geyser_city_state_location = find_images_to_find_in("geyser")
print("geyser done")
glacier_match_with_images,glacier_city_state_location = find_images_to_find_in("glacier")
print("glacier done")
road_match_with_images,road_city_state_location = find_images_to_find_in("road")
print("road done")
northern_lights_match_with_images,northern_lights_city_state_location = find_images_to_find_in("northern lights")
print("northern lights done")
waterfall_match_with_images,waterfall_city_state_location = find_images_to_find_in("waterfall")
print("waterfall done")
mountain_match_with_images,mountain_city_state_location = find_images_to_find_in("mountain")
print("mountain done")
sea_match_with_images,sea_city_state_location = find_images_to_find_in("sea")
print("sea done")

geyser done
glacier done
road done
northern lights done
waterfall done
mountain done
sea done


In [18]:
app_iceland_results_file = codecs.open("app_iceland_results.txt","w",encoding="utf-8-sig")

In [12]:
def find_top_three_images(img,match_with_images,city_state_location):
    match_with_images = list(match_with_images)
    distances = pd.Series(match_with_images).apply(lambda x: calculateDistance(img, x))
    indexes = (distances.nsmallest(n=3, keep='first').index)
    results = []
    for i,city in enumerate(pd.Series(city_state_location)[indexes]):
        results.append(city)
    return results

In [1]:
f = open(r"app_scrolling_window.txt")
locations = f.read()
matches = re.findall(r"(.*?),(.*)\n",locations)
count=0
for i,rec in enumerate(matches):
    count+=1
    print(count)
    location = rec[0]
    class_ = rec[1]
    img = image.load_img("app_iceland\\"+location, target_size=(299, 299))
    # Preprocessing the image
    img = image.img_to_array(img)
    if class_=="mountain":
        results = find_top_three_images(img, mountain_match_with_images,mountain_city_state_location)
    elif class_=="sea":
        results = find_top_three_images(img, sea_match_with_images,sea_city_state_location)
    elif class_=="road":
        results = find_top_three_images(img, road_match_with_images,road_city_state_location)
    elif class_=="waterfall":
        results = find_top_three_images(img, waterfall_match_with_images,waterfall_city_state_location)
    elif class_=="northern lights":
        results = find_top_three_images(img, northern_lights_match_with_images,northern_lights_city_state_location)
    elif class_=="geyser":
        results = find_top_three_images(img, geyser_match_with_images,geyser_city_state_location)
    elif class_=="glacier":
        results = find_top_three_images(img, glacier_match_with_images,glacier_city_state_location)
    else:
        results=["","",""]
    "|".join(results)
    match_string = location+"|"+class_+"|"+"|".join(results)+'\n'
    app_iceland_results_file.write(match_string)
app_iceland_results_file.close()

In [28]:
model = load_model('./top_layers_v2.iv3.hdf5')

In [2]:
instagram_images = []
instagram_locations=[]
instagram_loc = r"app_iceland\\"
for file in os.listdir(instagram_loc):
    im1 = Image.open(instagram_loc+file)
    im2 = im1.resize((299, 299), Image.NEAREST)
    test_image = np.array(im2)
    instagram_images.append(test_image)
    print(file)
    instagram_locations.append(file)

In [3]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from sklearn.preprocessing import OneHotEncoder
np.random.seed(42)
enc = OneHotEncoder(sparse=False)
categories = np.array(['geyser','glacier','road','northern lights','sea','mountain','people','waterfall']).reshape(8,1)
enc.fit_transform(categories)

Using TensorFlow backend.


In [33]:
predictions_enc = model.predict(np.array(instagram_images),verbose=1)
predictions_unenc = enc.inverse_transform(predictions_enc)

236/236 [==============================] - 184s 778ms/step


In [50]:
app_scrolling_window_file = open("app_iceland_results.txt","w")

In [4]:
count=0
for i in range(len(instagram_locations)):
    count+=1
    print(count)
    app_scrolling_window_file.write(instagram_locations[i]+","+predictions_unenc[i][0]+"\n")

In [51]:
app_scrolling_window_file.close()